In [2]:
import pandas as pd
import pathlib
import numpy as np
from numpy import log as ln
import statistics as stats

In [21]:
# Import dataset
data_dir = pathlib.Path(
    '../data')

metadata_file = pathlib.Path(data_dir, "plate_details_per_screen.parquet.gzip")
metadata_df = pd.read_parquet(metadata_file)
print(metadata_df.head())

In [6]:
# Define stats
# Relative frequencies
def rel_freq(attribute_elements):
    total_instances = sum(attribute_elements.values())
    p_dict = dict()
    for i in attribute_elements.keys():
        p_dict[i] = attribute_elements[i] / total_instances
    return p_dict
    
# Shannon Index
def h_index(p):
    """Calculates the Shannon Index of a set of unique attribute instances.
    Parameters
    ----------
    p: dict
        Dictionary of relative frequencies for each unique element in an attribute column.

    Returns
    -------
    h: float
        Shannon Index value
    
    results: list
        List of each -p_iln(p_i) value to use for Normalized Median Evenness statistic.
    """
    results = list()
    for entry in p.values():
        results.append(entry * ln(entry))
    
    h = -(sum(results))
    return h, results

# Pielou's Evenness
def pielou(h, s):
    return h / ln(s)

# Normalized Median Evenness
def nme(h_list):
    temp_list = list()
    for h_value in h_list:
        temp_list.append(-1.0 * h_value)
    nme = stats.median(temp_list) / np.max(temp_list)
    return nme

In [7]:
# Define stat collection workflow
def collect_stats(metadata_pq, na_cols=["pixel_size_x", "pixel_size_y"]):

    # Read parquet into pandas df
    metadata_df = pd.read_parquet(metadata_pq)
    study_name = metadata_pq.split('.')[0]
    attribute_names = metadata_df.columns.to_list()

    # Remove irrelevant attributes
    for attribute in na_cols:
        attribute_names.remove(attribute)
    attribute_results = list()

    # Collect statistics for each attribute
    for attribute in attribute_names:
        if attribute == "stain":
            pass
        elif attribute == "stain_target":
            pass
        else:
            unique_entries = metadata_df[attribute].unique()
            attribute_elements = dict()
            for element in unique_entries:
                attribute_elements[element] = len(metadata_df[metadata_df[attribute] == element])

            # Richness
            s = len(attribute_elements.keys())

            # Shannon Index
            p_dict = rel_freq(attribute_elements=attribute_elements)
            h, pi_list = h_index(p=p_dict)

            # Calculate Normalized Median Evenness
            nme_result = nme(pi_list)

            # Calculate Pielou's evenness
            j = pielou(h=h, s=s)

            # Append stats to attribute_results
            attribute_results.append([study_name,
                                    attribute, 
                                    s,
                                    h, 
                                    nme_result, 
                                    j])

    stat_results_df = pd.DataFrame(data=attribute_results, columns=['Study_Name', 'Attribute', 'S', 'H', 'NME', 'J'])

    return stat_results_df